# Part 1 - Data Preprocessing

### Importing the Libraries

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

import warnings
warnings.filterwarnings('ignore')

### Importing the Dataset

In [2]:
training_set = pd.read_csv('../input/users-movies-ratings/ml-100k/ml-100k/u1.base', delimiter='\t')
training_set = np.array(training_set, dtype='int')
test_set = pd.read_csv('../input/users-movies-ratings/ml-100k/ml-100k/u1.test', delimiter='\t')
test_set = np.array(test_set, dtype='int')

### Getting the number of users and movies

In [3]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

In [4]:
nb_users, nb_movies

(943, 1682)

### Converting the data into an array with users in lines and movies in columns

In [5]:
def convert(data):
    new_data=[]
    for id_users in range(1, nb_users+1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

training_set = convert(training_set)
test_set = convert(test_set)

### Converting the data into Torch Tensor

In [6]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

# Part 2 - Creating model Architecture and Training

In [7]:
class SAE(nn.Module):
    
    def __init__(self,):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20,10)
        self.fc3 = nn.Linear(10,20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
        
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

In [8]:
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr=0.01, weight_decay=0.5)

### Training the SAE( Stacked auto encoder)

In [9]:
nb_epoch = 200
for epoch in range(1, nb_epoch+1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        inputs = Variable(training_set[id_user]).unsqueeze(0)
        target = inputs.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(inputs)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data>0)+1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(1.7716)
epoch: 2 loss: tensor(1.0965)
epoch: 3 loss: tensor(1.0535)
epoch: 4 loss: tensor(1.0383)
epoch: 5 loss: tensor(1.0309)
epoch: 6 loss: tensor(1.0267)
epoch: 7 loss: tensor(1.0240)
epoch: 8 loss: tensor(1.0219)
epoch: 9 loss: tensor(1.0207)
epoch: 10 loss: tensor(1.0195)
epoch: 11 loss: tensor(1.0189)
epoch: 12 loss: tensor(1.0185)
epoch: 13 loss: tensor(1.0177)
epoch: 14 loss: tensor(1.0178)
epoch: 15 loss: tensor(1.0173)
epoch: 16 loss: tensor(1.0168)
epoch: 17 loss: tensor(1.0167)
epoch: 18 loss: tensor(1.0164)
epoch: 19 loss: tensor(1.0162)
epoch: 20 loss: tensor(1.0161)
epoch: 21 loss: tensor(1.0162)
epoch: 22 loss: tensor(1.0158)
epoch: 23 loss: tensor(1.0159)
epoch: 24 loss: tensor(1.0160)
epoch: 25 loss: tensor(1.0158)
epoch: 26 loss: tensor(1.0156)
epoch: 27 loss: tensor(1.0152)
epoch: 28 loss: tensor(1.0151)
epoch: 29 loss: tensor(1.0139)
epoch: 30 loss: tensor(1.0121)
epoch: 31 loss: tensor(1.0101)
epoch: 32 loss: tensor(1.0094)
epoch: 33 loss: t

# Part 3 - Test on test set

In [10]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    inputs = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user])
    if torch.sum(target.data > 0) >0:
        output = sae(inputs)
        target.require_grad = False
        output[(target == 0).unsqueeze(0)] = 0
        loss = criterion(output, target)
        mean_collector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))    

test loss: tensor(0.4324)
